# 데이터셋 처리 및 병합

사용 데이터:
1. 네이버 영화 리뷰 감성 분류
2. AI HUB 감성 대화 말뭉치

출처)
1. https://github.com/e9t/nsmc
2. https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86

특이사항)
1. 감성 대화 말뭉치는 슬픔,분노,불안 등의 감정을 모두 부정(0)으로 라벨링

2. 감성 대화 말뭉치는 긍정 66% 부정 33% 비율인 Imbalanced DATA이기 때문에 긍정 데이터를 랜덤으로 일부 제거

3. 네이버 영화 리뷰 감성 분류의 경우 너무 15만개라는 데이터가 방대하기 때문에 랜덤으로 5만개 추출

4. 최종적으로 부정문: 36282, 긍정문: 31831 대략 7만개의 데이터로 학습과 테스트에 이용 예정

# Train Data Set 1

In [115]:
import pandas as pd
import numpy as np
import urllib.request

In [116]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
train_data = pd.read_table('ratings_train.txt')

In [117]:
train_data1 = train_data.sample(n=50000)

In [118]:
train_data1 = train_data1[['document','label','id']]
train_data1 =  train_data1.drop(['id'],axis =1)
train_data1.columns = ['Sentences','Emotions']

In [119]:
print(len(train_data1))

50000


In [120]:
train_data1.head()

,Sentences,Emotions
42271,브루수 윌리스 그는 나에게 전설이다..,1
20045,왜 아무도 안달지 ㅋㅋ 난 잼있게봣다,1
75857,좀 심한듯.. 배우들이 불쌍해..ㅜㅜ,0
87115,장재인이 없는 결승전이라 시시한 경쟁이었음.,0
2944,그런데 이거 개봉도하네?,0


# Train Data Set 2

In [121]:
import csv
train_data2 = pd.read_csv('/content/6emotions_training_set.csv')
print(len(train_data2)) 

40879


In [122]:
cnt = 0
for i in range(len(train_data2['Emotions'])):
  if train_data2['Emotions'][i] in ['불안','당황','슬픔','분노','상처','불안 ']:
    cnt = cnt + 1
    if cnt == 3:
      train_data2['Emotions'][i] = 0
      cnt = 0
    else:
      train_data2 = train_data2.drop(index=i)

  else:
    train_data2['Emotions'][i] = 1

In [123]:
train_data2 = train_data2[['Sentences','Emotions']]

In [124]:
print(len(train_data2))

18113


In [125]:
train_data2.head()

,Sentences,Emotions
0,아내가 드디어 출산하게 되어서 정말 신이 나.,1
3,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,1
4,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,1
5,이제 곧 은퇴할 시기가 되었어. 내가 먼저 은퇴를 하고 육 개월 후에 남편도 은퇴를...,0
8,친구들이 나를 괴롭혀. 부모님과 선생님께 얘기했는데도 믿어주지 않아.,0


# 최종 확인 및 저장

In [131]:
ans =[0,0]
for i in train_data1['Emotions']:
  ans[i] = ans[i] + 1

for i in train_data2['Emotions']:
  ans[i] = ans[i] + 1
print("부정문: ",ans[0])
print("긍정문: ",ans[1])  

부정문:  36282
긍정문:  31831


In [132]:
result = pd.concat([train_data1,train_data2])

In [133]:
print(len(result.columns))

2


In [134]:
print(len(result))

68113


In [135]:
result.head()

,Sentences,Emotions
42271,브루수 윌리스 그는 나에게 전설이다..,1
20045,왜 아무도 안달지 ㅋㅋ 난 잼있게봣다,1
75857,좀 심한듯.. 배우들이 불쌍해..ㅜㅜ,0
87115,장재인이 없는 결승전이라 시시한 경쟁이었음.,0
2944,그런데 이거 개봉도하네?,0


In [136]:
result.to_csv('P_or_N.csv')